In [ ]:
import cv2
import h5py
import imageio
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from IPython.display import Image
from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import (
    Activation,
    Conv3D,
    MaxPooling3D,
    UpSampling3D,
)
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.compat.v1.logging import INFO, set_verbosity
from tensorflow.keras import mixed_precision


set_verbosity(INFO)

K.set_image_data_format("channels_first")


In [ ]:
import numpy as np
import tensorflow as tf
from pathlib import Path

def data_generator(X_paths, y_paths):
    for x_path, y_path in zip(X_paths, y_paths):
        X = np.load(x_path)['X']  # (4, 128, 128, 16)
        y = np.load(y_path)['y']  # (3, 128, 128, 16)

        X = np.transpose(X, (0, 3, 1, 2))  # (4, 16, 128, 128)
        y = np.transpose(y, (0, 3, 1, 2))  # (3, 16, 128, 128)

        X = tf.convert_to_tensor(X, dtype=tf.float32)
        y = tf.convert_to_tensor(y, dtype=tf.float32)
        yield X, y


def get_dataset(X_paths, y_paths, batch_size=2, shuffle=True):
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(X_paths, y_paths),
        output_types=(tf.float32, tf.float32),
        output_shapes=((4, 16, 128, 128), (3, 16, 128, 128))
    )
    if shuffle:
        dataset = dataset.shuffle(buffer_size=100)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset


In [ ]:
@tf.keras.utils.register_keras_serializable()
def soft_dice_loss(y_true, y_pred, axis=(2, 3, 4), epsilon=1e-7):
    """
    Compute the soft Dice loss (1 - mean Dice coefficient).

    Args:
        y_true (Tensor): Ground truth binary masks (B, C, D, H, W)
        y_pred (Tensor): Predicted masks (after sigmoid), same shape
        axis (tuple): Axes over which to compute Dice (D, H, W)
        epsilon (float): Small value to avoid division by zero

    Returns:
        Tensor: Scalar tensor (float32) representing soft Dice loss
    """
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    numerator = 2 * K.sum(y_true * y_pred, axis=axis) + epsilon
    denominator = K.sum(K.square(y_true), axis=axis) + K.sum(K.square(y_pred), axis=axis) + epsilon

    dice = numerator / denominator
    loss = 1 - K.mean(dice)
    
    return loss



In [ ]:
def create_convolution_block(input_layer, n_filters, batch_normalization=False,
                             kernel=(3, 3, 3), activation=None,
                             padding='same', strides=(1, 1, 1),
                             instance_normalization=False):
    """
    :param strides:
    :param input_layer:
    :param n_filters:
    :param batch_normalization:
    :param kernel:
    :param activation: Keras activation layer to use. (default is 'relu')
    :param padding:
    :return:
    """
    layer = Conv3D(n_filters, kernel, padding=padding, strides=strides, data_format="channels_first", dtype='float32')(
        input_layer)
    if activation is None:
        return Activation('relu')(layer)
    else:
        return activation()(layer)


def get_up_convolution(n_filters, pool_size, kernel_size=(2, 2, 2),
                       strides=(2, 2, 2),
                       deconvolution=False):
    if deconvolution:
        return Deconvolution3D(filters=n_filters, kernel_size=kernel_size,
                               strides=strides,dtype='float32')
    else:
        return UpSampling3D(size=pool_size,dtype='float32')


def unet_model_3d(loss_function, input_shape=(4, 16, 128, 128),
                  pool_size=(2, 2, 2), n_labels=3,
                  initial_learning_rate=0.00001,
                  deconvolution=False, depth=4, n_base_filters=32,
                  include_label_wise_dice_coefficients=False, metrics =[],
                  batch_normalization=False, activation_name="sigmoid"):
    """
    Builds the 3D UNet Keras model.f
    :param metrics: List metrics to be calculated during model training (default is dice coefficient).
    :param include_label_wise_dice_coefficients: If True and n_labels is greater than 1, model will report the dice
    coefficient for each label as metric.
    :param n_base_filters: The number of filters that the first layer in the convolution network will have. Following
    layers will contain a multiple of this number. Lowering this number will likely reduce the amount of memory required
    to train the model.
    :param depth: indicates the depth of the U-shape for the model. The greater the depth, the more max pooling
    layers will be added to the model. Lowering the depth may reduce the amount of memory required for training.
    :param input_shape: Shape of the input data (n_chanels, x_size, y_size, z_size). The x, y, and z sizes must be
    divisible by the pool size to the power of the depth of the UNet, that is pool_size^depth.
    :param pool_size: Pool size for the max pooling operations.
    :param n_labels: Number of binary labels that the model is learning.
    :param initial_learning_rate: Initial learning rate for the model. This will be decayed during training.
    :param deconvolution: If set to True, will use transpose convolution(deconvolution) instead of up-sampling. This
    increases the amount memory required during training.
    :return: Untrained 3D UNet Model
    """
    inputs = Input(input_shape,dtype='float32')
    current_layer = inputs
    levels = list()

    # add levels with max pooling
    for layer_depth in range(depth):
        layer1 = create_convolution_block(input_layer=current_layer,
                                          n_filters=n_base_filters * (
                                                  2 ** layer_depth),
                                          batch_normalization=batch_normalization)
        layer2 = create_convolution_block(input_layer=layer1,
                                          n_filters=n_base_filters * (
                                                  2 ** layer_depth) * 2,
                                          batch_normalization=batch_normalization)
        if layer_depth < depth - 1:
            current_layer = MaxPooling3D(pool_size=pool_size)(layer2)
            levels.append([layer1, layer2, current_layer])
        else:
            current_layer = layer2
            levels.append([layer1, layer2])

    # add levels with up-convolution or up-sampling
    for layer_depth in range(depth - 2, -1, -1):
        n_filters = K.int_shape(current_layer)[1]  # dla channels_first
        
        up_convolution = get_up_convolution(
            pool_size=pool_size,
            deconvolution=deconvolution,
            n_filters=n_filters
        )(current_layer)
    
        concat = concatenate(
            [up_convolution, levels[layer_depth][1]],
            axis=1  # channels_first = kanały na osi 1
        )
    
        level_filters = K.int_shape(levels[layer_depth][1])[1]
    
        current_layer = create_convolution_block(
            n_filters=level_filters,
            input_layer=concat,
            batch_normalization=batch_normalization
        )
        current_layer = create_convolution_block(
            n_filters=level_filters,
            input_layer=current_layer,
            batch_normalization=batch_normalization
        )


    final_convolution = Conv3D(
        n_labels,
        kernel_size=(1, 1, 1),
        activation=activation_name,
        data_format="channels_first",
        dtype='float32'
    )(current_layer)
    
    model = Model(inputs=inputs, outputs=final_convolution)

    return model


In [ ]:
@tf.keras.utils.register_keras_serializable()
def dice_mean(y_true, y_pred, axis=(2, 3, 4), epsilon=1e-7):
    """
    Compute the mean Dice coefficient across all classes and batch elements.
    Uses keras backend (K) for compatibility and performance in mixed precision.

    Args:
        y_true (Tensor): Ground truth masks with shape (B, C, D, H, W)
        y_pred (Tensor): Predicted masks (after sigmoid activation), same shape
        axis (tuple): Axes over which to compute sums (D, H, W)
        epsilon (float): Small constant to avoid division by zero

    Returns:
        Tensor: Mean Dice coefficient (scalar)
    """
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')

    numerator = 2 * K.sum(y_true * y_pred, axis=axis) + epsilon
    denominator = K.sum(K.square(y_true), axis=axis) + K.sum(K.square(y_pred), axis=axis) + epsilon

    dice = numerator / denominator  # shape: (B, C)
    return K.mean(dice)  # scalar: mean over batch and class


dice_mean.__name__ = "dice_mean"

In [ ]:
from pathlib import Path

data_dir = Path("/kaggle/input/braintumor-dataset-patches")
train_X = sorted(list((data_dir / "train" / "images").glob("*.npz")))
train_y = sorted(list((data_dir / "train" / "masks").glob("*.npz")))
val_X = sorted(list((data_dir / "val" / "images").glob("*.npz")))
val_y = sorted(list((data_dir / "val" / "masks").glob("*.npz")))


train_dataset = get_dataset(train_X, train_y)
val_dataset = get_dataset(val_X, val_y, shuffle=False)
train_dataset = train_dataset.repeat()
val_dataset = val_dataset.repeat()



for X_batch, y_batch in train_dataset.take(1):
    print(f"X batch shape: {X_batch.shape}, y batch shape: {y_batch.shape}")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger, TensorBoard


checkpoint_cb =ModelCheckpoint(
    filepath="/kaggle/working/checkpoints/unet3d_best.keras",
    monitor="val_dice_mean",
    mode="max",
    save_best_only=True,
    save_weights_only=False,
    verbose=1
)

earlystop_cb = EarlyStopping(
    monitor="val_dice_mean", mode="max", patience=10, restore_best_weights=True
)

reduce_lr_cb = ReduceLROnPlateau(
    monitor="val_dice_mean",
    factor=0.5,
    patience=5,
    cooldown=2,
    min_lr=1e-6,
    min_delta=1e-4,
    threshold_mode="rel",
    verbose=1
)


csv_logger = CSVLogger("/kaggle/working/training_log.csv", append=True)


callbacks = [checkpoint_cb, earlystop_cb, reduce_lr_cb,csv_logger]

In [ ]:
mixed_precision.set_global_policy("float32")
print("global precision policy:", mixed_precision.global_policy())

strategy = tf.distribute.MirroredStrategy()
print(f"Using GPU: {strategy.num_replicas_in_sync}")

with strategy.scope():
    model = unet_model_3d(
        loss_function=soft_dice_loss, 
        metrics=[dice_mean]
    )
    
    model.compile(
        optimizer=Adam(learning_rate=1e-4),
        loss=soft_dice_loss,
        metrics=[dice_mean]
    )


n_epochs=100
steps_per_epoch = len(train_X) // 2
validation_steps = len(val_X) // 2


history = model.fit(train_dataset,
        epochs=n_epochs,
        validation_data=val_dataset,
        steps_per_epoch=steps_per_epoch,
        validation_steps=validation_steps,
        callbacks = callbacks,
        verbose=1)